In [1]:
import numpy as np
import pandas as pd
import scipy.io as sio
import mne
from mne.datasets import fetch_fsaverage

In [247]:
cases = {1: ('A0R0', 'p200309', ['8B2-3', '8B3-4', '8B4-5', '8B7-8', '9C4-5']), #fixed!
        2: ('M0S0', 'p200723', ['6HC6-7', '6HC7-8', '6HC8-9', '8NA4-5', '11HC2-3']), #fixed!
        3: ('M0R0', 'p201209', ['1TP2-3', '2NA5-6', '3HT7-8']), #fixed!
        4: ('M0K0', 'p210831', ['1TP2-3', '2HC3-4', '3HT2-3', '3HT3-4', '3HT4-5']), #fixed!
        5: ('Z0M0', 'p220602', []), #fixed!
        6: ('E0G0', 'p200924', ['1AA1-2', '1AA2-3', '1AA3-4', '4DD1-2', '7GG2-3', '7GG3-4', '11KK6-7', '15OO5-6'])} #fixed!

In [710]:
CASE = cases[2][1]
MEG_CASE = cases[2][0]
seeg_mask = cases[2][2]
VOL_SOURCES = 8194

localization_file = fr'Z:\Diane_Vrubel\Data\sEEG\iEEG_localization.xlsx'
mat_file = fr'Z:\Diane_Vrubel\Data\sEEG\erf\erf_{MEG_CASE}.mat'

src_meg = fr'Z:\Diane_Vrubel\Data\MEG\{MEG_CASE}\forward_model\src.pckl'
trans_meg = mne.read_trans(fr'Z:\Diane_Vrubel\Data\MEG\{MEG_CASE}\forward_model\checked_visually_trans.fif')
iz_meg_file = fr'Z:\Diane_Vrubel\Data\MEG_predictions\{MEG_CASE}_pred_nobin_8.out'
subjects_dir_meg = fr'Z:\Diane_Vrubel\Data\FreeSurfer'

## sEEG

In [711]:
# paths to mne datasets - sample sEEG and FreeSurfer's fsaverage subject
# which is in MNI space
misc_path = mne.datasets.misc.data_path()
sample_path = mne.datasets.sample.data_path()
subjects_dir = sample_path / "subjects"

# use mne-python's fsaverage data
fetch_fsaverage(subjects_dir=subjects_dir, verbose=True)  # downloads if needed

0 files missing from root.txt in C:\Users\CCDM\mne_data\MNE-sample-data\subjects
0 files missing from bem.txt in C:\Users\CCDM\mne_data\MNE-sample-data\subjects\fsaverage


MNEWindowsPath('C:/Users/CCDM/mne_data/MNE-sample-data/subjects/fsaverage')

In [712]:
erf_data = sio.loadmat(mat_file)

In [713]:
erf_data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Sun Jul 16 13:25:35 2023',
 '__version__': '1.0',
 '__globals__': [],
 'erf_all': array([[(array([[ 0.08571415,  0.11428553,  0.05714276,  0.02857138,  0.17142829,
                  0.08571415,  0.25714244,  0.11428553,  0.14285691,  0.11428553,
                  0.02857138,  0.        ,  0.        ,  0.        ,  0.02857138,
                  0.02857138,  0.        ,  0.02857138,  0.17142829,  0.22857106,
                  0.54285626,  0.3142852 ,  0.02857138,  0.02857138,  0.05714276,
                  0.        ,  0.        ,  0.28571382,  0.37142797,  0.37142797,
                  0.94285561,  1.71428292,  0.34285658,  0.08571415,  0.11428553,
                  0.14285691,  0.34285658,  0.17142829,  9.54284161,  9.62855576,
                  1.85713983,  0.        ,  0.02857138,  0.05714276,  1.19999805,
                  0.68571317,  0.        ,  0.59999902,  0.17142829,  0.08571415,
                 13.59997786,

In [714]:
spike_rate = erf_data['erf_all'][0][0][1][0]  # Spike rate 
#hfo_rate = erf_data['erf_all'][0][0][4][0]  # HFO rate 
electrodes = erf_data['erf_all'][0][0][-1]  # names

localization_df = pd.read_excel(localization_file, sheet_name=CASE) 
localization_df.columns = ['Electrode', 'X', 'Y', 'Z', 'X_2', 'Y_2', 'Z_2', 'Region'] 
region_data = localization_df[['Electrode', 'Region']].set_index('Electrode')['Region'].to_dict()

In [715]:
electrodes

array([[array(['1LA1-2'], dtype='<U6')],
       [array(['1LA2-3'], dtype='<U6')],
       [array(['1LA3-4'], dtype='<U6')],
       [array(['1LA4-5'], dtype='<U6')],
       [array(['1LA5-6'], dtype='<U6')],
       [array(['2LB2-3'], dtype='<U6')],
       [array(['2LB3-4'], dtype='<U6')],
       [array(['2LB4-5'], dtype='<U6')],
       [array(['2LB5-6'], dtype='<U6')],
       [array(['2LB6-7'], dtype='<U6')],
       [array(['2LB7-8'], dtype='<U6')],
       [array(['3Pl1-2'], dtype='<U6')],
       [array(['3Pl2-3'], dtype='<U6')],
       [array(['3Pl3-4'], dtype='<U6')],
       [array(['3Pl4-5'], dtype='<U6')],
       [array(['3Pl5-6'], dtype='<U6')],
       [array(['3Pl6-7'], dtype='<U6')],
       [array(['3Pl7-8'], dtype='<U6')],
       [array(['3Pl8-9'], dtype='<U6')],
       [array(['3Pl9-10'], dtype='<U7')],
       [array(['3Pl10-11'], dtype='<U8')],
       [array(['3Pl11-12'], dtype='<U8')],
       [array(['4GC1-2'], dtype='<U6')],
       [array(['4GC2-3'], dtype='<U6')],
       [arr

In [716]:
electrodes[2][0][0]

array(['1LA3-4'], dtype='<U6')

In [717]:
# saving coordinates to a dictionary
electrode_coords = {}
for index, row in localization_df.iterrows():
    electrode_coords[row['Electrode']] = (row['X_2'], row['Y_2'], row['Z_2'])

In [718]:
spike_data = [] # a list of dictionaries with electrodes coords and their spike_rate

for i, electrode in enumerate(electrodes):
    electrode_name = electrode[0][0]  # Access the string and split it
   # if electrode_name in electrode_coords:
    coords_1 = electrode_coords[electrode_name.split('-')[0]]
    if electrode_name.split('-')[0][:(-1)] + str(int(electrode_name.split('-')[0][-1]) + 1) in electrode_coords:
        second_electrode = electrode_name.split('-')[0][:(-1)] + str(int(electrode_name.split('-')[0][-1]) + 1)
    elif electrode_name.split('-')[0][:(-1)] + electrode_name.split('-')[1][-1] in electrode_coords:
        second_electrode = electrode_name.split('-')[0][:(-1)] + electrode_name.split('-')[1][-1]
    else:
        second_electrode = electrode_name.split('-')[0]
    coords_2 = electrode_coords[second_electrode]
    spike_data.append({
            'Electrode': electrode_name,
            'X': (coords_1[0]+coords_2[0])/2,
            'Y': (coords_1[1]+coords_2[1])/2,
            'Z': (coords_1[2]+coords_2[2])/2,
            'Spike Rate': spike_rate[i]
        })
   # else:
    #    print(f"Electrode {electrode_name} not found in coordinates")

spike_df = pd.DataFrame(spike_data)
print("Spike DataFrame:")
print(spike_df)

Spike DataFrame:
   Electrode      X      Y      Z  Spike Rate
0     1LA1-2  -9.10  44.20  21.70    1.371426
1     1LA2-3 -14.35  46.30  23.15    5.142849
2     1LA3-4 -19.80  48.85  25.15    7.257131
3     1LA4-5 -24.85  51.00  27.35    2.114282
4     1LA5-6 -29.00  53.30  28.35    1.114284
..       ...    ...    ...    ...         ...
88   12HT5-6  51.65 -39.45   0.10    3.342852
89   12HT6-7  57.55 -40.75   0.45    0.771427
90   12HT7-8  63.25 -41.70   0.55    0.428571
91   12HT8-9  68.85 -42.45   0.60    0.828570
92  12HT9-10  74.35 -43.00   0.95    3.114281

[93 rows x 5 columns]


In [719]:
spike_df.head()

,Electrode,X,Y,Z,Spike Rate
0,1LA1-2,-9.10,44.20,21.70,1.371426
1,1LA2-3,-14.35,46.30,23.15,5.142849
2,1LA3-4,-19.80,48.85,25.15,7.257131
3,1LA4-5,-24.85,51.00,27.35,2.114282
4,1LA5-6,-29.00,53.30,28.35,1.114284


In [720]:
spike_df[spike_df.Electrode.isin(seeg_mask)]

,Electrode,X,Y,Z,Spike Rate
43,6HC6-7,-49.15,-15.80,-25.45,29.457095
44,6HC7-8,-54.60,-15.70,-25.05,0.000000
45,6HC8-9,-60.00,-15.15,-24.60,0.000000
55,8NA4-5,-27.95,-3.85,-26.95,44.342785
76,11HC2-3,30.80,-11.40,-17.85,0.000000


In [721]:
spike_df = spike_df[~spike_df.Electrode.isin(seeg_mask)]

In [722]:
spike_df.Electrode.values

array(['1LA1-2', '1LA2-3', '1LA3-4', '1LA4-5', '1LA5-6', '2LB2-3',
       '2LB3-4', '2LB4-5', '2LB5-6', '2LB6-7', '2LB7-8', '3Pl1-2',
       '3Pl2-3', '3Pl3-4', '3Pl4-5', '3Pl5-6', '3Pl6-7', '3Pl7-8',
       '3Pl8-9', '3Pl9-10', '3Pl10-11', '3Pl11-12', '4GC1-2', '4GC2-3',
       '4GC3-4', '4GC4-5', '4GC5-6', '4GC9-10', '4GC10-11', '4GC11-12',
       '5HT2-3', '5HT3-4', '5HT4-5', '5HT5-6', '5HT6-7', '5HT7-8',
       '5HT8-9', '5HT9-10', '6HC1-2', '6HC2-3', '6HC3-4', '6HC4-5',
       '6HC5-6', '6HC9-10', '7TB3-4', '7TB4-5', '7TB5-6', '7TB6-7',
       '7TB7-8', '8NA1-2', '8NA2-3', '8NA3-4', '8NA5-6', '8NA6-7',
       '8NA7-8', '8NA8-9', '8NA9-10', '9RR1-2', '9RR2-3', '9RR3-4',
       '9RR4-5', '9RR5-6', '9RR6-7', '9RR7-8', '9RR8-9', '9RR9-10',
       '10FT1-2', '10FT2-3', '10FT3-4', '10FT4-5', '10FT5-6', '11HC1-2',
       '11HC3-4', '11HC4-5', '11HC5-6', '11HC6-7', '11HC7-8', '11HC8-9',
       '11HC9-10', '12HT1-2', '12HT2-3', '12HT3-4', '12HT4-5', '12HT5-6',
       '12HT6-7', '12HT7-8', 

In [723]:
# finding the electrode with the highest spike activity

max_spike_electrode = spike_df.loc[spike_df['Spike Rate'].idxmax()]

print(f"Electrode with max spike rate: {max_spike_electrode['Electrode']}")
print(max_spike_electrode['Spike Rate'])
print(f"Localization: X={max_spike_electrode['X']}, Y={max_spike_electrode['Y']}, Z={max_spike_electrode['Z']}")

Electrode with max spike rate: 7TB4-5
55.85705194397239
Localization: X=-42.400000000000006, Y=2.1, Z=-35.45


In [724]:
MAX_SPIKE = max_spike_electrode['Spike Rate']

In [725]:
ch_names=list(electrode_coords.keys())
coords_mni_ras = np.array(localization_df[['X_2','Y_2','Z_2']])

montage = mne.channels.make_dig_montage(
    ch_pos=dict(zip(ch_names, coords_mni_ras/1000.0)),
    coord_frame='head'  # because MNI is in MRI space
)

info = mne.create_info(ch_names=ch_names, ch_types='seeg', sfreq=1000)
info.set_montage(montage)

<Info | 8 non-empty values
 bads: []
 ch_names: 1LA1, 1LA2, 1LA3, 1LA4, 1LA5, 1LA6, 2LB1, 2LB2, 2LB3, 2LB4, ...
 chs: 112 sEEG
 custom_ref_applied: False
 dig: 115 items (3 Cardinal, 112 EEG)
 highpass: 0.0 Hz
 lowpass: 500.0 Hz
 meas_date: unspecified
 nchan: 112
 projs: []
 sfreq: 1000.0 Hz
>

In [726]:
info

<Info | 8 non-empty values
 bads: []
 ch_names: 1LA1, 1LA2, 1LA3, 1LA4, 1LA5, 1LA6, 2LB1, 2LB2, 2LB3, 2LB4, ...
 chs: 112 sEEG
 custom_ref_applied: False
 dig: 115 items (3 Cardinal, 112 EEG)
 highpass: 0.0 Hz
 lowpass: 500.0 Hz
 meas_date: unspecified
 nchan: 112
 projs: []
 sfreq: 1000.0 Hz
>

In [727]:
identity_trans = mne.transforms.Transform('head', 'mri')  # head → mri
identity_trans['trans'] = np.eye(4)

view_kwargs = dict(azimuth=105, elevation=100, focalpoint=(0, 0, -15))
'''
brain = mne.viz.Brain(
    "fsaverage",
    subjects_dir=subjects_dir,
    cortex="low_contrast",
    alpha=0.25,
    background="white",
    # surf = 'inflated'
)

brain.add_sensors(info, trans=identity_trans)
brain.add_head(alpha=0.25, color="tan")
brain.show_view(distance=400, **view_kwargs)
'''

'\nbrain = mne.viz.Brain(\n    "fsaverage",\n    subjects_dir=subjects_dir,\n    cortex="low_contrast",\n    alpha=0.25,\n    background="white",\n    # surf = \'inflated\'\n)\n\nbrain.add_sensors(info, trans=identity_trans)\nbrain.add_head(alpha=0.25, color="tan")\nbrain.show_view(distance=400, **view_kwargs)\n'

In [728]:
# get standard fsaverage volume (5mm grid) source space
fname_src = subjects_dir / "fsaverage" / "bem" / "fsaverage-vol-5-src.fif"
vol_src = mne.read_source_spaces(fname_src)

aver_src = vol_src

    Reading a source space...
    [done]
    1 source spaces read


In [729]:
import numpy as np
import nibabel as nib
from scipy.spatial import cKDTree

In [730]:
# electrode MNI coordinates (the highest spike rate)
mni_coords_mm = np.array([[15.2, -6.6, -26.6]]) / 1000

# all source points from the source space
all_rr = np.concatenate([s['rr'][s['inuse'].astype(bool)] for s in vol_src])

tree = cKDTree(all_rr)
dist, idx = tree.query(mni_coords_mm) # the index of the closest source point

In [731]:
def find_closest(coords, tree):
    mni_coords_mm = np.array([coords]) # coords is a list of x y z
    dist, idx = tree.query(mni_coords_mm) # the index of the closest source point
    return idx

all_rr = np.concatenate([s['rr'][s['inuse'].astype(bool)] for s in vol_src])
tree = cKDTree(all_rr)

In [732]:
coordinates = [[x / 1000, y / 1000, z / 1000] for x, y, z in zip(spike_df.X.values, spike_df.Y.values, spike_df.Z.values)]
spike_rates = spike_df["Spike Rate"].values
src_coords = []
rates = dict()

for electrode, rate in zip(coordinates, spike_rates):
    src_id = find_closest(electrode, tree)
    src_coord = all_rr[src_id]
    if str(src_id) in rates:
        rates[str(src_id)].append(rate)
    else:
        rates[str(src_id)] = [rate]
    src_coords.append(src_coord)  

rates_ = dict([(src_id, sum(rates)/len(rates)) for src_id, rates in rates.items()])

In [733]:
all_rr.shape

(14629, 3)

In [734]:
all_rr_mm = all_rr 
source_coords_mm = all_rr_mm[idx]

In [735]:
source_coords_mm

array([[ 0.015, -0.005, -0.025]])

In [736]:
mni_coords_mm 

array([[ 0.0152, -0.0066, -0.0266]])

In [737]:
n_sources = all_rr.shape[0]
data = np.zeros((n_sources, 100))
for idx in rates.keys():
    idx_ = int(idx.replace('[', '').replace(']', ''))
    data[idx_] = np.array([rates_[idx]] * 100)

vertices = [np.arange(n_sources)]

In [738]:
idxs = [int(x.replace('[', '').replace(']', '')) for x in rates.keys()]

In [739]:
rates_tresh = dict()
rates_tresh_ = dict()
for key, value in rates.items():
    if value[0] / MAX_SPIKE > 0.3:
        rates_tresh[key] = value[0] / MAX_SPIKE * 3
        rates_tresh_[key] = value[0] / MAX_SPIKE * 3
    else:
        rates_tresh[key] = [0.0]
        rates_tresh_[key] = 0.0
    '''
    if value[0] >= 20:
        rates_tresh[key] = value[0]
        rates_tresh_[key] = value[0]
    else:
        rates_tresh[key] = [0.0]
        rates_tresh_[key] = 0.0
    rates_tresh[key] = [value[0] / 40.1]
    rates_tresh_[key] = value[0] / 40.1
    '''

In [740]:
unique_sorted_indices = np.array(sorted(idxs))
unique_spike_values = [rates_tresh_[f'[{idx}]'] for idx in unique_sorted_indices]
# Создаем данные для VolSourceEstimate
correct_vertices = np.array([vol_src[0]['vertno'][x] for x in sorted(idxs)])
vertices = [correct_vertices]  # Используем уникальные отсортированные индексы
data = np.array(unique_spike_values)[:, np.newaxis]  # Приведение к форме (n_vertices, n_times)

# Проверяем размерность и корректность
print(f"Data shape: {data.shape}, Vertices: {vertices}")

Data shape: (85, 1), Vertices: [array([11063, 11064, 11094, 11127, 12316, 12317, 12347, 12348, 13533,
       13537, 13538, 13540, 13541, 13542, 13605, 13607, 13608, 13609,
       14721, 14722, 14723, 14724, 14725, 14726, 14728, 14729, 14838,
       16160, 16161, 16162, 16196, 16197, 16198, 16199, 16441, 16474,
       16476, 16477, 16478, 17760, 17766, 17768, 17769, 19825, 19826,
       19828, 19829, 19830, 19854, 19856, 19857, 21523, 21524, 21525,
       21526, 21555, 23737, 24827, 24861, 25523, 26111, 26112, 26113,
       26809, 26840, 26841, 27394, 28126, 28724, 28725, 28750, 28751,
       28752, 28753, 28755, 28756, 28780, 28781, 28782, 30533, 30534,
       30535, 30537, 30538, 30539], dtype=int64)]


In [741]:
vals_to_verts = dict(zip(vertices[0].tolist(), unique_spike_values))

all_vertices = vol_src[0]['vertno']
all_values = []
for vert in all_vertices:
    if vert in vals_to_verts.keys():
        all_values.append(vals_to_verts[vert])
    else:
        all_values.append(0)
        
full_data = np.array(all_values)[:, np.newaxis]
full_verts = [all_vertices]

full_stc_seeg = mne.VolSourceEstimate(
    data=full_data,
    vertices=full_verts,
    tmin=0,
    tstep=1,
    subject="fsaverage"
)

In [742]:
iz_seeg = []
for act, vert in zip(data.tolist(), vertices[0].tolist()):
    if act[0] != 0:
        iz_seeg.append(vert)

In [743]:
len(iz_seeg)

19

In [744]:
vol_stc = mne.VolSourceEstimate(
    data=data,
    vertices=vertices,
    tmin=0,
    tstep=1,
    subject="fsaverage"
)

In [745]:
vol_stc

<VolSourceEstimate | 85 vertices, subject : fsaverage, tmin : 0.0 (ms), tmax : 0.0 (ms), tstep : 1000.0 (ms), data shape : (85, 1), ~2 kB>

In [746]:
mne.viz.set_3d_options(antialias=False)

In [747]:
clim = dict(kind="value", lims=[0.5, 0.7, 0.95])
"""
brain = vol_stc.plot_3d(
    src=vol_src,
    subjects_dir=subjects_dir,
    view_layout="horizontal",
    views=["axial", "coronal", "sagittal"],
    size=(800, 300),
    show_traces=0.4,
    clim=clim,
    add_data_kwargs=dict(colorbar_kwargs=dict(label_font_size=8)),
)
"""

'\nbrain = vol_stc.plot_3d(\n    src=vol_src,\n    subjects_dir=subjects_dir,\n    view_layout="horizontal",\n    views=["axial", "coronal", "sagittal"],\n    size=(800, 300),\n    show_traces=0.4,\n    clim=clim,\n    add_data_kwargs=dict(colorbar_kwargs=dict(label_font_size=8)),\n)\n'

In [748]:
aseg_file = fr'{subjects_dir}\fsaverage\mri\aparc+aseg.mgz'
label_names = mne.get_volume_labels_from_aseg(aseg_file)

In [749]:
stc_to_label = mne.extract_label_time_course(
    full_stc_seeg,
    (aseg_file, label_names),  
    vol_src,
    mode='max', 
    mri_resolution=True, 
    allow_empty=True 
)

peak_time_idx = np.argmax(np.max(stc_to_label, axis=0))
peak_activations = stc_to_label[:, peak_time_idx]

active_label_indices = np.where(peak_activations > 0)[0]
active_regions = [label_names[i] for i in active_label_indices]

regions_seeg = []
for region in active_regions:
    if ('Unknown' not in region) and ('unknown' not in region) and region != 'Left-Cerebral-White-Matter' and region != 'Right-Cerebral-White-Matter':
        regions_seeg.append(region)

print(len(regions_seeg))
print("Active volume regions:")
for region in regions_seeg:
    print(region)

Reading atlas C:\Users\CCDM\mne_data\MNE-sample-data\subjects\fsaverage\mri\aparc+aseg.mgz
114/114 atlas regions had at least one vertex in the source space
Extracting time courses for 114 labels (mode: max)
14
Active volume regions:
Left-Inf-Lat-Vent
Left-Cerebellum-Cortex
Left-Hippocampus
Left-Amygdala
ctx-lh-entorhinal
ctx-lh-fusiform
ctx-lh-inferiorparietal
ctx-lh-inferiortemporal
ctx-lh-middletemporal
ctx-lh-parahippocampal
ctx-lh-superiortemporal
ctx-lh-supramarginal
ctx-lh-temporalpole
ctx-lh-insula


In [750]:
'''
brain = mne.viz.Brain(
    "fsaverage",
    alpha=0.1,
    cortex="low_contrast",
    subjects_dir=subjects_dir
)
brain.add_volume_labels(aseg="aparc+aseg", labels=regions_seeg)
'''

'\nbrain = mne.viz.Brain(\n    "fsaverage",\n    alpha=0.1,\n    cortex="low_contrast",\n    subjects_dir=subjects_dir\n)\nbrain.add_volume_labels(aseg="aparc+aseg", labels=regions_seeg)\n'

## Forward Model

In [125]:
src_vol = mne.read_source_spaces(subjects_dir / "fsaverage" / "bem" / "fsaverage-vol-5-src.fif")
bem = mne.read_bem_solution(subjects_dir / "fsaverage" / "bem" / "fsaverage-5120-5120-5120-bem-sol.fif")

    Reading a source space...
    [done]
    1 source spaces read
Loading surfaces...

Loading the solution matrix...

Three-layer model surfaces loaded.
Loaded linear collocation BEM solution from C:\Users\CCDM\mne_data\MNE-sample-data\subjects\fsaverage\bem\fsaverage-5120-5120-5120-bem-sol.fif


In [126]:
src_vol[0]['vertno'].shape

(14629,)

In [38]:
montage = mne.channels.make_standard_montage('standard_1020')
ch_names = montage.ch_names[:64]
info = mne.create_info(ch_names=ch_names, sfreq=1000, ch_types='eeg')
info.set_montage(montage)

<Info | 8 non-empty values
 bads: []
 ch_names: Fp1, Fpz, Fp2, AF9, AF7, AF5, AF3, AF1, AFz, AF2, AF4, AF6, AF8, ...
 chs: 64 EEG
 custom_ref_applied: False
 dig: 67 items (3 Cardinal, 64 EEG)
 highpass: 0.0 Hz
 lowpass: 500.0 Hz
 meas_date: unspecified
 nchan: 64
 projs: []
 sfreq: 1000.0 Hz
>

In [39]:
fwd_vol = mne.make_forward_solution(
    info=info,
    trans='fsaverage',  
    src=src_vol,
    bem=bem,
    eeg=True,
    meg=False
)

Source space          : <SourceSpaces: [<volume, shape=(33, 39, 34), n_used=14629>] MRI (surface RAS) coords, subject 'fsaverage', ~75.3 MB>
MRI -> head transform : C:\Users\CCDM\AppData\Roaming\Python\Python312\site-packages\mne\data\fsaverage\fsaverage-trans.fif
Measurement data      : instance of Info
Conductor model   : instance of ConductorModel
Accurate field computations
Do computations in head coordinates
Free source orientations

Read 1 source spaces a total of 14629 active source locations

Coordinate transformation: MRI (surface RAS) -> head
    0.999994 0.003552 0.000202      -1.76 mm
    -0.003558 0.998389 0.056626      31.09 mm
    -0.000001 -0.056626 0.998395      39.60 mm
    0.000000 0.000000 0.000000       1.00

Read  64 EEG channels from info
Head coordinate coil definitions created.
Source spaces are now in head coordinates.

Employing the head->MRI coordinate transform with the BEM model.
BEM model instance of ConductorModel is now set up

Source spaces are in head

In [40]:
fwd_vol

EEG,64
Source space,Volume with 14629 grid points
Source orientation,Free


In [41]:
fwd_vol['sol']['data'].shape

(64, 43887)

In [42]:
43887 / 3

14629.0

In [43]:
src_vol[0]['vertno'].shape

(14629,)

In [44]:
gain = fwd_vol['sol']['data']
n_sources = gain.shape[1] // 3
gain_fixed = np.zeros((gain.shape[0], n_sources))

for i in range(n_sources):
    gx = gain[:, 3*i + 0]
    gy = gain[:, 3*i + 1]
    gz = gain[:, 3*i + 2]
    gain_fixed[:, i] = np.sqrt(gx**2 + gy**2 + gz**2)

In [45]:
gain_fixed.shape

(64, 14629)

In [46]:
unique_sorted_indices = np.array(sorted(src_vol[0]['vertno']))
unique_spike_values = []
for idx in unique_sorted_indices:
    if f'[{idx}]' in rates_tresh_.keys():
        unique_spike_values.append(rates_tresh_[f'[{idx}]'])
    else:
        unique_spike_values.append(0)
# Создаем данные для VolSourceEstimate
correct_vertices_ = np.array([vol_src[0]['vertno'][x] for x in sorted(idxs)])
vertices_ = [correct_vertices]  # Используем уникальные отсортированные индексы
data_ = np.array(unique_spike_values)[:, np.newaxis]  # Приведение к форме (n_vertices, n_times)

In [47]:
source_data = data_ #* (10 ** (-8))

In [48]:
rates_tresh_

{'[1442]': 2.705236907730673,
 '[1441]': 2.060349127182045,
 '[963]': 0.0,
 '[962]': 0.0,
 '[961]': 0.0,
 '[1437]': 0.0,
 '[1436]': 0.0,
 '[1848]': 1.4154613466334165,
 '[1874]': 1.541147132169576,
 '[1873]': 0.0,
 '[1872]': 0.0,
 '[1871]': 0.0,
 '[1896]': 0.0,
 '[1895]': 0.0,
 '[1894]': 0.0,
 '[1919]': 0.0,
 '[6055]': 0.0,
 '[6054]': 1.229925187032419,
 '[5252]': 0.0,
 '[5251]': 1.020448877805486,
 '[5250]': 0.0,
 '[5249]': 0.0,
 '[5248]': 0.0,
 '[5247]': 0.0,
 '[5246]': 0.0,
 '[9151]': 0.0,
 '[9150]': 0.0,
 '[9149]': 0.0,
 '[9121]': 0.0,
 '[9120]': 0.0,
 '[9119]': 0.0,
 '[9118]': 0.0,
 '[9117]': 0.0,
 '[9116]': 0.0,
 '[9418]': 0.0,
 '[9417]': 0.0,
 '[10148]': 0.0,
 '[10147]': 0.0,
 '[10171]': 0.0,
 '[10875]': 0.0,
 '[10874]': 0.0,
 '[11534]': 0.0,
 '[8761]': 0.0,
 '[8760]': 0.0,
 '[8782]': 0.0,
 '[8781]': 0.0,
 '[8780]': 0.0,
 '[8801]': 0.0,
 '[8800]': 0.0,
 '[2590]': 2.3042394014962593,
 '[2592]': 1.8389027431421443,
 '[2593]': 1.7925187032418952,
 '[1987]': 0.9635910224438903,
 '[1

In [49]:
sensor_data = gain_fixed @ source_data

In [50]:
sensor_data

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [51]:
evoked = mne.EvokedArray(sensor_data, info=info, tmin=0)

In [52]:
from mne import make_ad_hoc_cov
noise_cov = make_ad_hoc_cov(info)

In [53]:
inverse_operator = mne.minimum_norm.make_inverse_operator(info, fwd_vol, noise_cov=noise_cov)

Computing inverse operator with 64 channels.
    64 out of 64 channels remain after picking
Selected 64 channels
Creating the depth weighting matrix...
    64 EEG channels
    limit = 14630/14629 = 3.882442
    scale = 61579.2 exp = 0.8
Whitening the forward solution.
Computing rank from covariance with rank=None
    Using tolerance 5.7e-18 (2.2e-16 eps * 64 dim * 0.0004  max singular value)
    Estimated rank (eeg): 64
    EEG: rank 64 computed from 64 data channels with 0 projectors
    Setting small EEG eigenvalues to zero (without PCA)
Creating the source covariance matrix
Adjusting source covariance matrix.
Computing SVD of whitened and weighted lead field matrix.


C:\Users\CCDM\AppData\Local\Temp\ipykernel_3928\2053878520.py:1: RuntimeWarning: No average EEG reference present in info["projs"], covariance may be adversely affected. Consider recomputing covariance using with an average eeg reference projector added.
  inverse_operator = mne.minimum_norm.make_inverse_operator(info, fwd_vol, noise_cov=noise_cov)
C:\Users\CCDM\AppData\Local\Temp\ipykernel_3928\2053878520.py:1: RuntimeWarning: No average EEG reference present in info["projs"], covariance may be adversely affected. Consider recomputing covariance using with an average eeg reference projector added.
  inverse_operator = mne.minimum_norm.make_inverse_operator(info, fwd_vol, noise_cov=noise_cov)


    largest singular value = 4.72127
    scaling factor to adjust the trace = 5.96068e+22 (nchan = 64 nzero = 0)


In [54]:
snr = 3.0
lambda2 = 1.0 / snr ** 2
evoked.set_eeg_reference(projection=True)
stc_cortex = mne.minimum_norm.apply_inverse(evoked, inverse_operator, lambda2=lambda2, method='dSPM')

EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    The projection vectors do not apply to these channels.
    Created the whitener using a noise covariance matrix with rank 64 (0 small eigenvalues omitted)
    Computing noise-normalization factors (dSPM)...
[done]
Applying inverse operator to ""...
    Picked 64 channels from the data
    Computing inverse...
    Eigenleads need to be weighted ...
    Computing residual...
    Explained   nan% variance
    Combining the current components...
    dSPM...
[done]


C:\Users\CCDM\AppData\Roaming\Python\Python312\site-packages\mne\minimum_norm\inverse.py:1041: RuntimeWarning: invalid value encountered in scalar divide
  var_exp = 1 - ((res * res.conj()).sum().real / (data * data.conj()).sum().real)


In [55]:
stc_cortex.data

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [56]:
brain = stc_cortex.plot_3d(
    src=vol_src,
    subjects_dir=subjects_dir,
    view_layout="horizontal",
    views=["axial", "coronal", "sagittal"],
    size=(800, 300),
    show_traces=0.4,
    clim=clim,
    add_data_kwargs=dict(colorbar_kwargs=dict(label_font_size=8)),
)

## MEG IZ

In [752]:
import pickle

with open(src_meg, 'rb') as f:
    src = pickle.load(f)
    
mne_coords = np.concatenate([s['rr'][s['inuse'].astype(bool)] for s in src])

In [753]:
#vol_mne = mne_coords[VOL_SOURCES::]

In [754]:
identity_trans = mne.transforms.Transform('head', 'mri')  # head → mri
identity_trans['trans'] = np.eye(4)

In [755]:
mni_coords = mne.head_to_mni(mne_coords, MEG_CASE, trans_meg, subjects_dir=subjects_dir_meg )

In [756]:
#mni_vol = mne.head_to_mri(vol_mne, MEG_CASE, trans_meg, subjects_dir=subjects_dir_meg)

In [757]:
activity = np.loadtxt(iz_meg_file)
data = activity[:, np.newaxis]

In [758]:
vol_activity = activity[VOL_SOURCES::]

In [759]:
src_coords = []

for source, rate in zip(mni_coords/1000, activity):
    src_id = find_closest(source, tree)
    src_coord = all_rr[src_id]
    if rate == 1 and src_id.tolist()[0] not in src_coords:
        src_coords.append(src_id.tolist()[0])
correct_vertices = np.array([vol_src[0]['vertno'][x] for x in sorted(src_coords)])
vertices = [correct_vertices]  # Используем уникальные отсортированные индексы

In [760]:
data = np.ones((vertices[0].shape[0],1))

In [761]:
iz_meg = vertices[0].tolist()

In [762]:
all_vertices = vol_src[0]['vertno']
all_values_meg = []
for vert in all_vertices:
    if vert in correct_vertices.tolist():
        all_values_meg.append(3)
    else:
        all_values_meg.append(0)
full_data_meg = np.array(all_values_meg)[:, np.newaxis]
full_verts_meg = [all_vertices]

full_stc_meg = mne.VolSourceEstimate(
    data=full_data_meg,
    vertices=full_verts_meg,
    tmin=0,
    tstep=1,
    subject="fsaverage"
)

In [763]:
meg_to_label = mne.extract_label_time_course(
    full_stc_meg,
    (aseg_file, label_names),  
    vol_src,
    mode='max', 
    mri_resolution=True, 
    allow_empty=True 
)

peak_time_idx = np.argmax(np.max(meg_to_label, axis=0))
peak_activations = meg_to_label[:, peak_time_idx]

active_label_indices = np.where(peak_activations > 0)[0]
active_regions_meg = [label_names[i] for i in active_label_indices]

regions_meg = []
for region in active_regions_meg:
    if ('Unknown' not in region) and ('unknown' not in region) and region != 'Left-Cerebral-White-Matter' and region != 'Right-Cerebral-White-Matter':
        regions_meg.append(region)

print(len(regions_meg))
print("Active volume regions:")
for region in regions_meg:
    print(region)

Reading atlas C:\Users\CCDM\mne_data\MNE-sample-data\subjects\fsaverage\mri\aparc+aseg.mgz
114/114 atlas regions had at least one vertex in the source space
Extracting time courses for 114 labels (mode: max)
17
Active volume regions:
Left-Inf-Lat-Vent
Left-Pallidum
Left-Hippocampus
Left-Amygdala
Left-VentralDC
Right-Inf-Lat-Vent
Right-Hippocampus
Right-Amygdala
Right-VentralDC
ctx-lh-entorhinal
ctx-lh-middletemporal
ctx-lh-parahippocampal
ctx-lh-parsopercularis
ctx-lh-parstriangularis
ctx-lh-precentral
ctx-lh-rostralmiddlefrontal
ctx-lh-superiortemporal


In [764]:
brain = mne.viz.Brain(
    "fsaverage",
    alpha=0.1,
    cortex="low_contrast",
    subjects_dir=subjects_dir
)
brain.add_volume_labels(aseg="aparc+aseg", labels=regions_meg)

    Smoothing by a factor of 0.9


In [765]:
iz_stc = mne.VolSourceEstimate(
    data=data,
    vertices=vertices,
    tmin=0,
    tstep=1,
    subject="fsaverage"
)

In [766]:
iz_stc

<VolSourceEstimate | 152 vertices, subject : fsaverage, tmin : 0.0 (ms), tmax : 0.0 (ms), tstep : 1000.0 (ms), data shape : (152, 1), ~3 kB>

In [ ]:

clim = dict(kind="value", lims=[0.5, 0.7, 0.95])

brain = iz_stc.plot_3d(
    src=vol_src,
    subjects_dir=subjects_dir,
    view_layout="horizontal",
    views=["axial", "coronal", "sagittal"],
    size=(800, 300),
    show_traces=0.4,
    clim=clim,
    add_data_kwargs=dict(colorbar_kwargs=dict(label_font_size=8)),
)


## Metrics

In [767]:
interc = []

In [768]:
for region in regions_seeg:
    if region in regions_meg:
        interc.append(region)

for region in regions_meg:
    if region in regions_seeg and region not in interc:
        interc.append(region)

In [769]:
for region in interc:
    print(region)

Left-Inf-Lat-Vent
Left-Hippocampus
Left-Amygdala
ctx-lh-entorhinal
ctx-lh-middletemporal
ctx-lh-parahippocampal
ctx-lh-superiortemporal


In [770]:
union = set(regions_meg + regions_seeg)

In [771]:
len(interc)

7

In [772]:
len(union)

24

In [773]:
len(interc)/len(union)

0.2916666666666667

In [774]:
diff = set(regions_meg).difference(interc)

In [775]:
len(diff)

10

In [776]:
for elem in sorted(diff):
    print(elem)

Left-Pallidum
Left-VentralDC
Right-Amygdala
Right-Hippocampus
Right-Inf-Lat-Vent
Right-VentralDC
ctx-lh-parsopercularis
ctx-lh-parstriangularis
ctx-lh-precentral
ctx-lh-rostralmiddlefrontal


In [709]:
for vert in iz_seeg:
    if vert in iz_meg:
        interc.append(vert)

for vert in iz_meg:
    if vert in iz_seeg and vert not in interc:
        interc.append(vert)

In [144]:
union = set(iz_seeg + iz_meg)

In [145]:
len(union)

197

In [146]:
sim = len(interc) / len(union)

In [147]:
sim

0.0

In [148]:
len(interc) / len(iz_seeg)

0.0

In [149]:
len(iz_seeg)

17

In [150]:
iz_seeg

[11127,
 12282,
 12317,
 13541,
 13542,
 13606,
 13608,
 13610,
 14723,
 14838,
 14872,
 16161,
 21061,
 26809,
 27396,
 28750,
 30535]